In [1]:
import tensorflow as tf
import tensorflow_io as tfio

In [2]:
from IPython.display import Audio

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from ipywidgets import interact

# Fonctions

In [4]:
from birds.data.spectrogram import full_spectro_generation, generate_mel_spectrogram, generate_db_scale_mel_spectrogram

# Dataset's spectrograms ?

In [5]:
RAW_DATA_PATH = '../raw_data/'
TRAIN_SET_PATH = '../raw_data/data_30s/train/'

In [6]:
metadata_train = pd.read_csv(RAW_DATA_PATH + 'metadata_train.csv')
metadata_train.head(3)

,Recording_ID,Genus,Specific_epithet,Subspecies,English_name,Recordist,Country,Locality,Latitude,Longitude,...,Other_species24,Other_species25,Other_species26,Other_species27,Other_species28,Other_species29,Other_species30,Species,Path,Target
0,447407,Sonus,naturalis,NaN,Soundscape,José Carlos Sires,Spain,"Córdoba, Córdoba, Andalucía",37.9413,-4.8958,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Sonus naturalis,Sonus-naturalis-447407_tens.ogg,0
1,387437,Sonus,naturalis,NaN,Soundscape,José Carlos Sires,Spain,"el planerón, belchite, zaragoza, aragón",41.2784,-0.7328,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Sonus naturalis,Sonus-naturalis-387437_tens.ogg,0
2,383228,Sonus,naturalis,NaN,Soundscape,José Carlos Sires,Spain,"río guadalmellato, córdoba, andalucía",38.0306,-4.6698,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Sonus naturalis,Sonus-naturalis-383228_tens.ogg,0


In [7]:
species_table = metadata_train.copy()[['Species', 'English_name', 'Target']]
species_table = species_table.drop_duplicates()
species_table = species_table.reset_index(drop=True)
species_table.head(3)

,Species,English_name,Target
0,Sonus naturalis,Soundscape,0
1,Fringilla coelebs,Common Chaffinch,1
2,Parus major,Great Tit,2


In [8]:
y_train = pd.read_csv(TRAIN_SET_PATH + 'y_train.csv')
y_train.head(3)

,Path,Target
0,Sonus-naturalis-447407_tens.ogg,0
1,Sonus-naturalis-387437_tens.ogg,0
2,Sonus-naturalis-383228_tens.ogg,0


In [18]:
@interact(target=range(50), record=range(32))
def print_from_target(target, record):
    
    files_names = list(y_train[y_train['Target'] == target].reset_index(drop=True).loc[record:record+2, 'Path'])
    files_paths = [TRAIN_SET_PATH + file_name for file_name in files_names]
    
    plt.figure(figsize=(20,5), constrained_layout=True)

    for j in range(2):
        file_name = files_names[j]
        file_path = files_paths[j]
        record_name = file_name.split("_tens.ogg")[0]
        
        spectrogram = full_spectro_generation(file_path, target, split=True)[0]

        plt.subplot(1, 4, 2*j+1)
        plt.imshow((spectrogram).numpy())
        plt.axis('off')
        plt.title(f"Record n°{record + j}\n{record_name} [spectr.]\nshape: {spectrogram.shape}", fontsize=12, loc='left')

        plt.subplot(1, 4, 2*j+2)
        plt.imshow(tf.math.log(spectrogram).numpy())
        plt.axis('off')
        plt.title(f"Record n°{record + j}\n{record_name} [log spectr.]\nshape: {spectrogram.shape}", fontsize=12, loc='left')
    
    species_name = species_table[species_table['Target'] == target].loc[:, 'Species'].reset_index(drop=True)[0]
    english_name = species_table[species_table['Target'] == target].loc[:, 'English_name'].reset_index(drop=True)[0]

    plt.suptitle(f"Target {target}: {species_name.upper()} ~ {english_name}", fontsize=20);


interactive(children=(Dropdown(description='target', options=(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14…

In [13]:
@interact(target=range(50), record=range(32))
def display_audio_from_target(target, record):
    
    files_names = list(y_train[y_train['Target'] == target].reset_index(drop=True).loc[:, 'Path'])
    files_paths = [TRAIN_SET_PATH + file_name for file_name in files_names]
    
    return Audio(files_paths[record])

interactive(children=(Dropdown(description='target', options=(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14…

In [ ]:
n_rows = 2

plt.figure(figsize=(15, 3*n_rows), constrained_layout=True)

for i in range(n_rows):
    for j in range(5):
        plt.subplot(n_rows, 5, 5*i + j + 1)
        plot_spectrogram(sub_path_list[5*i + j])